# Setup

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from torchmetrics import Accuracy
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR

from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn import metrics

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import random
from glob import glob
from tqdm import tqdm
from scipy.io import loadmat

import torch
from torch import nn
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from datetime import datetime

In [3]:
from IPython.display import clear_output
import os, sys, shutil
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm, trange
from glob import glob
import random
import cv2 as cv
import torch.nn.functional as F
import json
from sklearn.preprocessing import LabelEncoder

In [4]:
# main_data_dir =  "/media/mountHDD2"
print(os.getcwd())
save_dir = os.getcwd()
os.chdir("..")
os.chdir("..")
main_dir = os.getcwd() 
print(os.listdir(main_dir))
print(save_dir)

/media/mountHDD2/linh/git/HeartResearch/Chapman Dataset/2D_model/Other_model
['Reimplement', 'Data_Explore', 'Diagnostics.xlsx', '.ipynb_checkpoints', '2D_model', '1D_Model', '2D_Loss']
/media/mountHDD2/linh/git/HeartResearch/Chapman Dataset/2D_model/Other_model


In [5]:
data_dir = "/media/mountHDD3/data_storage/biomedical_data/ecg_data/ECGDataDenoised"
# data_dir = "/media/mountHDD3/data_storage/biomedical_data/ecg_data/CPSC2018"
label_file = main_dir + "/Diagnostics.xlsx"
# label_file = "/media/mountHDD3/data_storage/biomedical_data/ecg_data/khoibaocon/single_label.csv"

In [6]:
ls = []
for file in glob(data_dir + "/*"):
    ls.append(file.split("/")[-1].split(".")[0].split("_")[0])
ls_list = pd.Series(ls)
ls_list.value_counts()

MUSE    10588
dtype: int64

In [7]:
# ls = []
# for file in glob(data_dir + "/*"):
#     ls.append(type(pd.read_csv(file, header=None)))
# ls_list = pd.Series(ls)
# ls_list.value_counts()

In [8]:
# os.listdir(data_dir)

In [9]:
# data_df = pd.read_csv(data_dir + "/MUSE_20180120_121711_19000.csv")
label_df = pd.read_excel(label_file)
label_df = label_df[['FileName', 'Rhythm']]
# label_df = label_df.drop([4419,1280, 3182,3740,3970,6602,7888,7969,8470,8472,8475,8701,9670,9711,10548,10555])
# label_df = label_df.drop([4419])
label_df

,FileName,Rhythm
0,MUSE_20180120_121711_19000,SA
1,MUSE_20180120_121704_86000,SA
2,MUSE_20180113_125357_13000,SA
3,MUSE_20180113_134825_04000,SB
4,MUSE_20180115_123455_79000,SB
...,...,...
10641,MUSE_20181222_204246_47000,SVT
10642,MUSE_20180115_120332_79000,SA
10643,MUSE_20180712_152507_30000,AF
10644,MUSE_20180118_181350_17000,SA


In [10]:
label_encoder = LabelEncoder()
label_df['Rhythm'] = label_encoder.fit_transform(label_df['Rhythm'])
# label_df['First_label'] = label_encoder.fit_transform(label_df['First_label'])
print(label_df)

                         FileName  Rhythm
0      MUSE_20180120_121711_19000       5
1      MUSE_20180120_121704_86000       5
2      MUSE_20180113_125357_13000       5
3      MUSE_20180113_134825_04000       7
4      MUSE_20180115_123455_79000       7
...                           ...     ...
10641  MUSE_20181222_204246_47000      10
10642  MUSE_20180115_120332_79000       5
10643  MUSE_20180712_152507_30000       0
10644  MUSE_20180118_181350_17000       5
10645  MUSE_20180116_121646_28000       9

[10646 rows x 2 columns]


In [11]:
filenames = label_df["FileName"].values.tolist()
# print((filenames))
drop_ls = ['MUSE_20180113_124215_52000', 'MUSE_20180119_174843_24000', 'MUSE_20180712_152022_92000', 
          'MUSE_20180712_152024_00000', 'MUSE_20180712_151353_58000', 'MUSE_20180114_080214_06000',
          'MUSE_20180210_130454_71000', 'MUSE_20180712_151351_36000','MUSE_20180712_152114_47000',
          'MUSE_20180113_180425_75000','MUSE_20180712_153140_95000','MUSE_20180712_152014_31000',
          'MUSE_20180712_151357_86000','MUSE_20180113_181145_89000','MUSE_20180712_153632_30000',
          'MUSE_20180120_121805_89000','MUSE_20180114_124230_39000','MUSE_20180712_152019_73000']
# c = 0
# for name in filenames:
#     if name in drop_ls:
#         print(name)
#         print(c)
#     c += 1
# print(len(filenames))

In [12]:
unique_values = np.unique(label_df["Rhythm"].values.tolist())
print(unique_values)

[ 0  1  2  3  4  5  6  7  8  9 10]


In [13]:
data_paths = []
for file in glob(data_dir +"/*"):
    data_paths.append(file)
# data_paths = [data_dir + f"/{x}.csv" for x in filenames if x not in drop_ls]
print(len(data_paths))
# print(data_paths[0])
# print(data_paths[0].split("/")[-1].split(".")[0])
# data_test = pd.read_csv(data_paths[0], header = None)
# print(type(data_test.values.T))

10588


In [14]:
# data_dir = "/media/mountHDD2/khoibaocon"
# print(os.listdir(data_dir))
print(type(label_df["Rhythm"][0]))

<class 'numpy.int64'>


In [15]:
# main_df = pd.read_csv(data_dir + "/Label.csv")
# main_df.shape

In [16]:
# single_main_df = main_df[main_df["Second_label"].isnull()]
# single_fns = single_main_df["Recording"].values.tolist()
# single_mat_paths = [data_dir + f"/alldata/{x}.mat" for x in single_fns]

In [17]:
ratio = [0.9, 0.1]

train_index = int(len(data_paths)*ratio[0])
valid_index = int(len(data_paths)*(ratio[0]+ratio[1]))
print(train_index)
train_mat_paths = data_paths[:train_index]
valid_mat_paths = data_paths[train_index:valid_index]
print(len(train_mat_paths))

train_label = label_df.iloc[:train_index,:]
valid_label = label_df.iloc[train_index:valid_index,:]

9529
9529


In [18]:
print(len(train_label))
print(len(valid_label))

9529
1059


In [19]:
label_df[label_df["FileName"] == filenames]["Rhythm"].values

array([5, 5, 5, ..., 0, 5, 9])

In [20]:
label_train = []
for file in train_mat_paths:
    
    filename = file.split("/")[-1].split(".")[0]
    label = label_df[label_df["FileName"] == filename]["Rhythm"].values.item()
    label_train.append(label)

cw_dict = {}
for i in range(len(np.unique(np.array(label_train)))):
    cw_dict[i] = label_train.count(i)

print(cw_dict)

{0: 396, 1: 1600, 2: 107, 3: 15, 4: 8, 5: 356, 6: 7, 7: 3480, 8: 1655, 9: 1423, 10: 482}


 # Data Loader

In [21]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.nn.functional import one_hot
import math

In [22]:
class HeartData(Dataset):
    def __init__(self, data_paths, label_df):
        self.data_paths = data_paths
        random.shuffle(self.data_paths)
        self.label_df = label_df
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
        self.transform = transforms.Compose([
            transforms.Resize((224, 224), antialias=None),
            normalize
        ])

    def __getitem__(self, idx):
        data_path = self.data_paths[idx]        
        # data = loadmat(data_path)['ECG'][0][0][2]
        data = pd.read_csv(data_path, header = None)
        data = data.values.T
        clip_data = data[:, 500:3000]
        clip_data = torch.tensor(clip_data, dtype=torch.float32)

        filename = data_path.split("/")[-1].split(".")[0]
        label = self.label_df[self.label_df["FileName"] == filename]["Rhythm"].values.item()

        # return torch_data_resize, label-1
        return clip_data, label

    def __len__(self):
        return len(self.data_paths)  

def get_class_weight(labels_dict):
    total = sum(labels_dict.values())
    max_num = max(labels_dict.values())
    mu = 1.0 / (total / max_num)
    class_weight = dict()
    for key, value in labels_dict.items():
        score = math.log(mu * total / float(value))
        class_weight[key] = score if score > 1.0 else 1.0
    return class_weight

In [23]:
train_ds = HeartData(train_mat_paths, label_df)
valid_ds = HeartData(valid_mat_paths, label_df)

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu", index = 1)
batch_size = 128

traindl = DataLoader(
    train_ds,
    batch_size=batch_size, 
    shuffle=True, 
    pin_memory=True, 
    num_workers=os.cpu_count()//2
)

validdl = DataLoader(
    valid_ds,
    batch_size=1, 
    # shuffle=True, 
    pin_memory=True, 
    num_workers=os.cpu_count()//2
)

# Model 

In [25]:
class supervised():
    def __init__(self):
        super(supervised, self).__init__()
        self.train_params = {
            'num_epochs': 60,
            'batch_size': 128,
            'weight_decay': 1e-4,
            'learning_rate': 1e-3,
            'feature_dim': 1*128
        }

class chapman():
    def __init__(self):
        super(mit, self).__init__()
        # data parameters
        self.num_classes = 11
        self.class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
        self.sequence_len = 186

        # model configs
        self.input_channels = 12
        self.kernel_size = 8
        self.stride = 1
        self.dropout = 0.2

        # features
        self.mid_channels = 32
        self.final_out_channels = 128

        # Transformer
        self.trans_dim = 25
        self.num_heads = 5

    def get_dataset_class(dataset_name):
        """Return the algorithm class with the given name."""
        if dataset_name not in globals():
            raise NotImplementedError("Dataset not found: {}".format(dataset_name))
        return globals()[dataset_name]

In [26]:
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights
from torch import nn

class ecgTransForm(nn.Module):
    def __init__(self):
        
        super(ecgTransForm, self).__init__()

        filter_sizes = [5, 9, 11]
        self.num_classes = 11
        self.class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
        self.sequence_len = 186

        # model configs
        self.input_channels = 12
        self.kernel_size = 8
        self.stride = 1
        self.dropout = 0.2

        # features
        self.mid_channels = 32
        self.final_out_channels = 128

        # Transformer
        self.trans_dim = 315
        self.num_heads = 5
        self.conv1 = nn.Conv1d(self.input_channels, self.mid_channels, kernel_size=filter_sizes[0],
                               stride=self.stride, bias=False, padding=(filter_sizes[0] // 2))
        self.conv2 = nn.Conv1d(self.input_channels, self.mid_channels, kernel_size=filter_sizes[1],
                               stride=self.stride, bias=False, padding=(filter_sizes[1] // 2))
        self.conv3 = nn.Conv1d(self.input_channels, self.mid_channels, kernel_size=filter_sizes[2],
                               stride=self.stride, bias=False, padding=(filter_sizes[2] // 2))

        self.bn = nn.BatchNorm1d(self.mid_channels)
        self.relu = nn.ReLU()
        self.mp = nn.MaxPool1d(kernel_size=2, stride=2, padding=1)
        self.do = nn.Dropout(self.dropout)

        self.conv_block2 = nn.Sequential(
            nn.Conv1d(self.mid_channels, self.mid_channels * 2, kernel_size=8, stride=1, bias=False, padding=4),
            nn.BatchNorm1d(self.mid_channels * 2),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2, padding=1)
        )
        self.conv_block3 = nn.Sequential(
            nn.Conv1d(self.mid_channels * 2, self.final_out_channels, kernel_size=8, stride=1, bias=False,
                      padding=4),
            nn.BatchNorm1d(self.final_out_channels),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2, padding=1),
        )
        self.inplanes = 128
        self.crm = self._make_layer(SEBasicBlock, 128, 3)

        self.encoder_layer = nn.TransformerEncoderLayer(d_model=self.trans_dim, nhead=self.num_heads, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=3)

        self.aap = nn.AdaptiveAvgPool1d(1)
        self.clf = nn.Linear(1*128, self.num_classes)

    def _make_layer(self, block, planes, blocks, stride=1):  # makes residual SE block
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv1d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(planes * block.expansion),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x_in):
        # Multi-scale Convolutions
        
        x1 = self.conv1(x_in)
        # print("x1: ", x1.shape)
        x2 = self.conv2(x_in)
        # print("x2: ", x2.shape)
        x3 = self.conv3(x_in)
        # print("x3: ", x3.shape)

        x_concat = torch.mean(torch.stack([x1, x2, x3],2), 2)
        # print("x_concat: ", x_concat.shape)
        x_concat = self.do(self.mp(self.relu(self.bn(x_concat))))
        # print("x_concat: ", x_concat.shape)

        x = self.conv_block2(x_concat)
        # print("x: ", x.shape)
        x = self.conv_block3(x)
        # print("x: ", x.shape)

        # Channel Recalibration Module
        x = self.crm(x)
        # print("x: ", x.shape)

        # Bi-directional Transformer
        x1 = self.transformer_encoder(x)
        # print("x1: ", x1.shape)
        x2 = self.transformer_encoder(torch.flip(x,[2]))
        # print("x1: ", x2.shape)
        x = x1+x2
        # print("x: ", x.shape)

        x = self.aap(x)
        x_flat = x.reshape(x.shape[0], -1)
        x_out = self.clf(x_flat)
        return x_out


class SELayer(nn.Module):
    def __init__(self, channel, reduction=4):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1)
        return x * y.expand_as(x)

class SEBasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None,
                 *, reduction=4):
        super(SEBasicBlock, self).__init__()
        self.conv1 = nn.Conv1d(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm1d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(planes, planes, 1)
        self.bn2 = nn.BatchNorm1d(planes)
        self.se = SELayer(planes, reduction)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.se(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

model = ecgTransForm()
x = torch.randn((1, 12, 2500))
y = torch.randint(0, 10, (1,))
print(y.shape)
out = model(x)
print(nn.CrossEntropyLoss()(out, y))

torch.Size([1])
tensor(2.3466, grad_fn=<NllLossBackward0>)


# Training

In [27]:
epoch = 60
lr = 1e-3 # lr = 0.001 Acc: 0.821875 lr = 0.0005 Acc: 0.825 Focal_loss Acc: 0.828
best_acc = 0
best_ep = 0
weight_decay = 1e-4
feature_dim = 1*128

model.to(device)
optimizer = Adam(
            model.parameters(),
            lr=lr,
            weight_decay=weight_decay,
            betas=(0.9, 0.99))
scheduler = CosineAnnealingLR(optimizer=optimizer, T_max=epoch*len(traindl))

In [28]:
class FocalClassifierV0(nn.Module):
    def __init__(self, gamma=0.3):
        super().__init__()
        
        self.gamma = gamma
        self.act = nn.LogSoftmax(dim=1)
    
    def forward(self, pred, target):

        logits = self.act(pred)

        B, C = tuple(logits.size())

        entropy = torch.pow(1 - logits, self.gamma) * logits * F.one_hot(target, num_classes=C).float()

        return (-1 / B) * torch.sum(entropy)

focalloss_fn = FocalClassifierV0()
loss_fn = nn.CrossEntropyLoss()
checkpoint_folder = "run_ECGTransform"

In [29]:
def checkpoint(valid_class_acc, 
               val_total_loss,
               old_valid_class_acc,
               old_valid_loss,
               epoch, 
               model,
               optimizer,
               check_folder
#                    logs
              ):

    if valid_class_acc >= old_valid_class_acc and val_total_loss <= old_valid_loss:
        old_valid_class_acc = valid_class_acc
        old_valid_loss = val_total_loss
        save_dict = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': val_total_loss,
            'test_acc': valid_class_acc
        }

     # Saving best model
        now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
        run_dir = save_dir + f"/{check_folder}"
        if not os.path.exists(run_dir):
            os.mkdir(run_dir)
        save_best_model_dir = run_dir + "/save_best_model"
        if not os.path.exists(save_best_model_dir):
            os.mkdir(save_best_model_dir)
        save_best_model_path = save_best_model_dir + f"/{save_dict['loss']:>7f}_{save_dict['test_acc']:>7f}_{now}.pt"
        torch.save(save_dict, save_best_model_path)
        
def classification_report_csv(report, auc, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = save_dir + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    save_report_dir = run_dir + "/save_classification_report"
    if not os.path.exists(save_report_dir):
        os.mkdir(save_report_dir)
        
    report_data = report['macro avg']
    del report_data['support']
    report_data.update({'auc': auc})
    with open(save_report_dir + f"/cls_report_{now}.json", "w") as outfile: 
        json.dump(report_data, outfile)
    
def acc_loss_json(log_dict, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = save_dir + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    save_json_dir = run_dir + "/save_acc_loss_json"
    if not os.path.exists(save_json_dir):
        os.mkdir(save_json_dir) 
    save_acc_loss_file = save_json_dir + f"/acc_loss_{now}.json"
    with open(save_acc_loss_file, "w") as outfile: 
        json.dump(log_dict, outfile)

In [30]:
log_dict = {
    "train": {
        "acc": [],
        "loss": []
    },
    "valid": {
        "acc": [],
        "loss": []
    }
}

In [31]:
cls_weight = get_class_weight(cw_dict)
weights = [float(value) for value in cls_weight.values()]
weights_array = np.array(weights).astype(np.float32)  # Ensuring the correct dtype
weights_tensor = torch.tensor(weights_array).to(device)
loss_fn = nn.CrossEntropyLoss(weight=weights_tensor)

In [32]:
# cls_weight = get_class_weight(cw_dict)
# weights = [value for value in cls_weight.values()]
# weights_array = np.array(weights)  # Ensuring the correct dtype
# weights_tensor = torch.tensor(weights_array).to(device)
# loss_fn = nn.CrossEntropyLoss()

In [33]:
# Training

# class_la = np.unique(label_df["Rhythm"].values.tolist())
class_la = []
for i in range (11):
    class_la.append(i)
for i in range (len(class_la)):
    class_la[i] = str(class_la[i])
train_losses = []
train_acc_plot = []
val_losses = []
val_acc_plot = []
old_valid_class_acc = 0
old_valid_loss = 1e23
for e in range(epoch):
    model.train()
    print(f"Epoch: {e+1}")
    batch_cnt = 0
    total_loss = 0
    correct = 0
    for batch, (train_sig, train_label) in tqdm(enumerate(traindl)):
        batch_cnt = batch
        train_sig = train_sig.to(device=device)
        # print(train_sig.shape)
        train_label = train_label.to(device=device)    
        pred = model(train_sig)
        loss = loss_fn(pred, train_label)
 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        scheduler.step()
        total_loss += loss.item()
        correct += (pred.argmax(1) == train_label).type(torch.float).sum().item()
        
    total_loss /= batch_cnt
    correct /= len(traindl.dataset)
    log_dict["train"]["loss"].append(total_loss)
    log_dict["train"]["acc"].append(correct)
    
    print(f"train loss: {total_loss} - train acc: {100*correct}")

# Valid
    batch_cnt = 0
    val_total_loss = 0
    val_correct = 0
    model.eval()
    y_true_list = [] 
    pred_list = []
    with torch.no_grad():
        for batch, (valid_sig, valid_label) in tqdm(enumerate(validdl)):
            batch_cnt = batch
            valid_label = valid_label.to(device=device)
            
            valid_sig = valid_sig.to(device=device) 
            pred = model(valid_sig)
            pred_pos = pred.argmax(1)
            
            y_true_list.append(valid_label)
            pred_list.append(pred_pos)
            
            loss = loss_fn(pred, valid_label)
            val_total_loss += loss.item()
            val_correct += (pred.argmax(1) == valid_label).type(torch.float).sum().item()
            
        val_total_loss /= batch_cnt
        val_correct /= len(validdl.dataset)
        log_dict["valid"]["loss"].append(val_total_loss)
        log_dict["valid"]["acc"].append(val_correct)

        if val_correct > best_acc:
            best_acc = val_correct
            best_ep = e 

        print(f"valid loss: {val_total_loss} - valid acc: {100*val_correct}")
#         checkpoint(valid_class_acc = val_correct, 
#                    val_total_loss = val_total_loss,
#                    old_valid_class_acc = old_valid_class_acc ,
#                    old_valid_loss = old_valid_loss,
#                    epoch = e, 
#                    model = model,
#                    optimizer = optimizer,
#                    check_folder = checkpoint_folder)
        if val_correct >= old_valid_class_acc:
            old_valid_class_acc = val_correct
            old_valid_loss = val_total_loss
            save_dict = {
                'epoch': e,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': val_total_loss,
                'test_acc': val_correct
            }

         # Saving best model
            now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
            run_dir = save_dir + f"/{checkpoint_folder}"
            if not os.path.exists(run_dir):
                os.mkdir(run_dir)
            save_best_model_dir = run_dir + "/save_best_model"
            if not os.path.exists(save_best_model_dir):
                os.mkdir(save_best_model_dir)
            save_best_model_path = save_best_model_dir + f"/{save_dict['loss']:>7f}_{save_dict['test_acc']:>7f}_{now}.pt"
            torch.save(save_dict, save_best_model_path)
        
print(f"Best acuracy: {best_acc} at epoch {best_ep}")

y_true = torch.cat(y_true_list).cpu().numpy()
pred = torch.cat(pred_list).cpu().numpy()

# Save loss and acc to json file
acc_loss_json(log_dict, check_folder = checkpoint_folder)
    
fpr, tpr, thresholds = metrics.roc_curve(y_true, pred, pos_label = 0)

auc1 = metrics.auc(fpr, tpr)

reports = classification_report(y_true, pred, output_dict=True) 

classification_report_csv(report = reports, auc = auc1, check_folder = checkpoint_folder)

Epoch: 1


0it [00:00, ?it/s]

1it [00:03,  3.11s/it]

2it [00:03,  1.34s/it]

3it [00:03,  1.28it/s]

...

1000it [00:04, 205.84it/s]

1022it [00:04, 209.20it/s]

1046it [00:05, 217.67it/s]

1059it [00:05, 201.83it/s]

valid loss: 0.6639793514036829 - valid acc: 80.73654390934844
Best acuracy: 0.8158640226628895 at epoch 52


# Evaluation